In [1]:
import requests
import json
import pandas as pd
import shutil
import pathlib
import datetime
import pymongo
import datetime
from bson.json_util import dumps, loads

In [2]:
# This function gets data from web API and converts to csv file.
def get_jokes():
    columns = ['id','date','type', 'setup','punchline']
    jokes_df = pd.DataFrame(columns=columns)
    now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    now_data = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
    for i in range(3):
        response = requests.get('https://official-joke-api.appspot.com/random_joke')
        jokes = json.loads(response.text)
        new_row = {'id':jokes['id'], 'date':now_data,'type':jokes['type'], 'setup':jokes['setup'], 'punchline':jokes['punchline']}
        jokes_df = jokes_df.append(new_row, ignore_index=True)
    jokes_df.to_csv('data/temp/jokes_{}.csv'.format(now), sep=',', index=False)
    original = str(pathlib.Path().absolute()) + '\\data\\temp\\jokes_{}.csv'.format(now)
    target = str(pathlib.Path().absolute()) + '\\data\\input\\jokes_{}.csv'.format(now)
    shutil.copyfile(original, target)
    print('New CSV file created.')
    return

In [11]:
get_jokes()

New CSV file created.


In [4]:
# This creates a client that uses the default port on localhost.
client = pymongo.MongoClient()
db = client.humour

In [10]:
# This brings data from collection of humour.jokes in JSON format.
cursor = db.jokes.find({}, {'_id':0})

# Data can be filtered like this
#cursor = db.jokes.find({"date": "2021-05-22 15:38:22"}, {'_id':0})

cursor_data = list(cursor) 
json_data = dumps(cursor_data, indent = 2) 
print(json_data)

[
  {
    "date": "2021-05-22 16:22:55",
    "jokes": [
      {
        "id_": 157,
        "type": "2021-05-22 16:22:15",
        "setup": "general",
        "punchline": "What are the strongest days of the week?"
      },
      {
        "id_": 103,
        "type": "2021-05-22 16:22:15",
        "setup": "general",
        "punchline": "Did you know crocodiles could grow up to 15 feet?"
      },
      {
        "id_": 113,
        "type": "2021-05-22 16:22:15",
        "setup": "general",
        "punchline": "Have you heard the rumor going around about butter?"
      }
    ]
  }
]


In [6]:
# This creates JSON file as an ouput from MongoDB
with open('data/output/jokes.json', 'w') as file:
    file.write(json_data)

In [7]:
# This can be used to delete data from collection of humour.jokes
col = db["jokes"]
x = col.delete_many({})